- here put your symbol, data range, it will import data from nse and then convert it into monthaly timeframe and then save it to monthaly_data/symbol.csv 
- then it perform 5-10 lacs calculation and find max profit in given date range and save this found data to 'monthaly_all_in_one.csv'
- then add cumsum and cumprod in monthaly_data/symbol.csv and save it
- it not works for indexs

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from nselib import capital_market 
import pandas as pd

import matplotlib.pyplot as plt 
symbol = "ADANIENT"
fdate='01-01-2014'
tdate='25-08-2024'


## importing data

In [10]:

data = capital_market.price_volume_and_deliverable_position_data(symbol=symbol, from_date=fdate, to_date=tdate)
df = pd.DataFrame(data)



In [11]:
df1 = df[["Date", "Symbol", "OpenPrice", "HighPrice", "LowPrice", "ClosePrice"]]

In [ ]:
def replace_commas(x):
    if isinstance(x, str):
        return x.replace(',', '')
    return x

# Apply the function to columns
df1["HighPrice"] = df1['HighPrice'].apply(replace_commas)
try:
    df1["OpenPrice"] = df1['OpenPrice'].apply(replace_commas)
    df1["ClosePrice"] = df1['ClosePrice'].apply(replace_commas)
    df1["LowPrice"] = df1['LowPrice'].apply(replace_commas)
except:
    print("Replace done!")

# Convert columns to numeric
df1["HighPrice"] = pd.to_numeric(df1["HighPrice"], errors='coerce')
df1["OpenPrice"] = pd.to_numeric(df1["OpenPrice"], errors='coerce')
df1["ClosePrice"] = pd.to_numeric(df1["ClosePrice"], errors='coerce')
df1["LowPrice"] = pd.to_numeric(df1["LowPrice"], errors='coerce')

In [ ]:
df1['Date']=pd.to_datetime(df1['Date'])
df1["day"] = df1['Date'].dt.day
df1["month"] = df1['Date'].dt.month
df1["year"] = df1['Date'].dt.year
df1=df1.groupby(["year","month"])

## converting in monthaly time frame and preparing for calculation

In [14]:
month=[]
year=[]
OpenPrice=[]
ClosePrice=[]
HighPrice=[]
LowPrice=[]
symbole=[]

for name , group in df1:   
   symbole.append(group['Symbol'].values[0])
   month.append(name[1])
   year.append(name[0])
   OpenPrice.append(group['OpenPrice'].values[0])
   ClosePrice.append(group['ClosePrice'].values[-1:][0])
   HighPrice.append(group['HighPrice'].max())
   LowPrice.append(group['LowPrice'].min()) 



In [15]:
df2={"symbol":symbole,"month":month,"year":year,"OpenPrice":OpenPrice,"HighPrice":HighPrice,"LowPrice":LowPrice,"ClosePrice":ClosePrice}
df2=pd.DataFrame(df2)
df1=df2

In [16]:


# Calculate percentage changes
df1["High%"] = (df1["HighPrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
df1["High%"] = df1["High%"].apply(lambda x: "{:.3f}".format(x))

df1["Low%"] = (df1["LowPrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
df1["Low%"] = df1["Low%"].apply(lambda x: "{:.3f}".format(x))

df1["Close%"] = (df1["ClosePrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
df1["Close%"] = df1["Close%"].apply(lambda x: "{:.3f}".format(x))

# Convert percentage columns back to numeric
df1["Low%"] = pd.to_numeric(df1["Low%"], errors='coerce')
df1["High%"] = pd.to_numeric(df1["High%"], errors='coerce')
df1["Close%"] = pd.to_numeric(df1["Close%"], errors='coerce')

# Calculate target and stoploss percentages
df1.loc[abs(df1['High%']) <= abs(df1['Low%']), 'target_wrt_h&l%'] = abs(df1['Low%'])
df1.loc[abs(df1['High%']) > abs(df1['Low%']), 'target_wrt_h&l%'] = abs(df1['High%'])
df1.loc[abs(df1['High%']) >= abs(df1['Low%']), 'stoploss_wrt_h&l%'] = abs(df1['Low%'])
df1.loc[abs(df1['High%']) < abs(df1['Low%']), 'stoploss_wrt_h&l%'] = abs(df1['High%'])

# Determine profit decider
df1.loc[abs(df1['target_wrt_h&l%']) == abs(df1['High%']), 'profit_decider'] = abs(df1['Low%'])
df1.loc[abs(df1['target_wrt_h&l%']) == abs(df1['Low%']), 'profit_decider'] = abs(df1['High%'])
df1.to_csv(f"monthaly_data/{symbol}.csv",index=False)

## function for the calculation

In [2]:
def prfdata(symbol,stploss,trgt):
    df1=pd.read_csv(f"monthaly_data/{symbol}.csv")   
    df1['stploss']=df1['target_wrt_h&l%'].apply(lambda x:stploss)
    df1['trgt']=df1['target_wrt_h&l%'].apply(lambda x:trgt)
    df1['stploss']=pd.to_numeric(df1['stploss'])
    df1['trgt']=pd.to_numeric(df1['trgt'])

    df1.loc[((abs(df1['target_wrt_h&l%']) >= df1['trgt']) & ((abs(df1['profit_decider'])) < df1['stploss'])), 'profit'] = (df1['trgt']-df1['stploss'])
    df1.loc[(abs(df1['profit_decider']) >= df1['stploss']) , 'profit'] = -(2 * df1['stploss'])
    df1.loc[((abs(df1['target_wrt_h&l%']) < abs(df1['trgt'])) & ((abs(df1['profit_decider'])) < df1['stploss']) ) , 'profit'] = (abs(df1['Close%'])-df1['stploss'])
    df1['profit'] = pd.to_numeric(df1['profit'])

    profit = df1['profit'].sum()
    # print(f"Total profit = {profit}% in 2 years")
    data1 = df1.groupby(['year']).agg({"profit": "sum"})

    data = 1 + data1.values / 100
    overallprofit = 100 * data.cumprod()[-1:][0]
    formatted_overallprofit = "{:.4f}".format(overallprofit)
    # print(f"Total profit made in 2 years = {formatted_overallprofit}%")

    arr=[]

    for i in range(len(data1.values)):
        n=data1.values[i][0]
        arr.append(n)
    dic={"tprofit":profit,"cprofit":formatted_overallprofit,"data":data1}
    # df1.to_csv(f"monthaly_data/{symbol}.csv",index=False)
    
    return dic


In [ ]:
def prfdata1(symbol,stploss,trgt):
    df1=pd.read_csv(f"monthaly_data/{symbol}.csv")   
    df1['stploss']=df1['target_wrt_h&l%'].apply(lambda x:stploss)
    df1['trgt']=df1['target_wrt_h&l%'].apply(lambda x:trgt)
    df1['stploss']=pd.to_numeric(df1['stploss'])
    df1['trgt']=pd.to_numeric(df1['trgt'])

    df1.loc[((abs(df1['target_wrt_h&l%']) >= df1['trgt']) & ((abs(df1['profit_decider'])) < df1['stploss'])), 'profit'] = (df1['trgt']-df1['stploss'])
    df1.loc[(abs(df1['profit_decider']) >= df1['stploss']) , 'profit'] = -(2 * df1['stploss'])
    df1.loc[((abs(df1['target_wrt_h&l%']) < abs(df1['trgt'])) & ((abs(df1['profit_decider'])) < df1['stploss']) ) , 'profit'] = (abs(df1['Close%'])-df1['stploss'])
    df1['profit'] = pd.to_numeric(df1['profit'])

    profit = df1['profit'].sum()
    # print(f"Total profit = {profit}% in 2 years")
    data1 = df1.groupby(['year']).agg({"profit": "sum"})

    data = 1 + data1.values / 100
    overallprofit = 100 * data.cumprod()[-1:][0]
    formatted_overallprofit = "{:.4f}".format(overallprofit)
    # print(f"Total profit made in 2 years = {formatted_overallprofit}%")

    arr=[]

    for i in range(len(data1.values)):
        n=data1.values[i][0]
        arr.append(n)
    dic={"tprofit":profit,"cprofit":formatted_overallprofit,"data":data1}
    df1.to_csv(f"monthaly_data/{symbol}.csv",index=False)
    
    return dic


In [ ]:
stp = np.arange(0,10,0.01)
trgt =np.arange(0,30,0.1)

max=0
fstp=0
ftrgt=0
pmdata=[]
for estp in stp:
    print(estp)
    for etrgt in trgt:
        cc=prfdata(symbol,estp,etrgt)        
        if(max<cc['tprofit']):            
            max=cc['tprofit']
            fstp=estp
            ftrgt=etrgt
            pmdata=cc['data']
            # print(f"{max}  {estp}  {etrgt} ")
prfdata1(symbol,fstp,ftrgt)
        
            


In [ ]:
maxc=(np.array(pmdata)/100+1).cumprod()[-1:][0]*100
dataf={"symbol":[symbol],"tprofit":[max],"cprofit":[maxc],"target":[ftrgt],"stoploss":[fstp]}
df23=pd.DataFrame(dataf)
df24=pd.read_csv("month_all_in_one.csv")
df25=pd.concat([df24,df23])
df25.to_csv("month_all_in_one.csv",index=False)

In [21]:

ddata=pd.read_csv(f"monthaly_data/{symbol}.csv")
ddata["cumsum"]=ddata['profit'].cumsum()
ddata["cumprod"]=(ddata['profit']/100+1).cumprod()
ddata.to_csv(f"monthaly_data/{symbol}.csv")